<a href="https://colab.research.google.com/github/Aditya40104/YBI-Internship-Project/blob/main/Bank_Customer_Churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 865.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler  # Updated to RandomOverSampler
import gradio as gr

# Load the dataset
df = pd.read_csv('https://github.com/YBI-Foundation/Dataset/raw/refs/heads/main/Bank%20Churn%20Modelling.csv')

# Preprocessing the dataset
df.set_index('CustomerId', inplace=True)
df.drop('Surname', axis=1, inplace=True)

# Encoding categorical variables
df['Geography'] = df['Geography'].replace({'France': 2, 'Germany': 1, 'Spain': 0})
df['Gender'] = df['Gender'].replace({'Male': 0, 'Female': 1})
df['Num Of Products'] = df['Num Of Products'].replace({1: 0, 2: 1, 3: 1, 4: 1})

# Creating a new feature for customers with zero bank balance
df['zero_balance'] = np.where(df['Balance'] == 0, 1, 0)

# Splitting the data into features and target variable
X = df.drop('Churn', axis=1)
y = df['Churn']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Handling imbalanced data using random oversampling
ros = RandomOverSampler(random_state=42)
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)

# Training the SVM model
best_svm_model = SVC(C=1, gamma=0.1, kernel="rbf", class_weight="balanced", probability=True)
best_svm_model.fit(X_train_ros, y_train_ros)

# Function to plot churn distribution
def plot_churn_distribution():
    plt.figure(figsize=(6, 4))
    sns.countplot(x='Churn', data=df, palette='Set2')
    plt.title('Churn Distribution')
    plt.xlabel('Churn (0 = Not Churn, 1 = Churn)')
    plt.ylabel('Count')
    plt.tight_layout()
    plt.savefig('churn_distribution.png')
    return 'churn_distribution.png'

# Function to make predictions based on user input
def predict_churn(CreditScore, Geography, Gender, Age, Tenure, Balance, NumOfProducts, HasCreditCard, IsActiveMember, EstimatedSalary):
    # Input validation
    if not (0 <= CreditScore <= 950):
        return "Error: Credit Score must be between 0 and 950.", None, None
    if not (0 <= Age <= 120):
        return "Error: Age must be between 0 and 120.", None, None
    if not (0 <= Tenure <= 40):
        return "Error: Tenure must be between 0 and 40.", None, None
    if Balance < 0:
        return "Error: Balance cannot be negative.", None, None
    if NumOfProducts not in [1, 2]:
        return "Error: Number of Products must be 1 or 2.", None, None

    try:
        # Prepare the input data as a DataFrame
        input_data = pd.DataFrame({
            'CreditScore': [CreditScore],
            'Geography': [Geography],
            'Gender': [Gender],
            'Age': [Age],
            'Tenure': [Tenure],
            'Balance': [Balance],
            'Num Of Products': [NumOfProducts],
            'Has Credit Card': [HasCreditCard],
            'Is Active Member': [IsActiveMember],
            'Estimated Salary': [EstimatedSalary],
            'zero_balance': [1 if Balance == 0 else 0]
        })

        # Encode categorical variables
        input_data['Geography'] = input_data['Geography'].replace({'France': 2, 'Germany': 1, 'Spain': 0})
        input_data['Gender'] = input_data['Gender'].replace({'Male': 0, 'Female': 1})
        input_data['Num Of Products'] = input_data['Num Of Products'].replace({1: 0, 2: 1})

        # Scale the input data using the same scaler
        input_data_scaled = scaler.transform(input_data)

        # Make prediction
        prediction = best_svm_model.predict(input_data_scaled)[0]
        proba = best_svm_model.predict_proba(input_data_scaled)[0][1]  # Probability of churn

        # Plot probability as a bar
        plt.figure(figsize=(5, 3))
        plt.bar(['Not Churn', 'Churn'], [1-proba, proba], color=['green', 'red'])
        plt.ylim(0, 1)
        plt.title('Churn Prediction Probability')
        plt.xlabel('Prediction')
        plt.ylabel('Probability')
        plt.tight_layout()
        plt.savefig('churn_probability.png')

        # Return the prediction result
        if prediction == 1:
            return "The customer is likely to churn.", 'churn_probability.png', plot_churn_distribution()
        else:
            return "The customer is not likely to churn.", 'churn_probability.png', plot_churn_distribution()
    except Exception as e:
        return f"Error in prediction: {e}", None, None

# Gradio interface
inputs = [
    gr.Number(label="Credit Score"),
    gr.Dropdown(choices=["France", "Germany", "Spain"], label="Geography"),
    gr.Dropdown(choices=["Male", "Female"], label="Gender"),
    gr.Number(label="Age"),
    gr.Number(label="Tenure"),
    gr.Number(label="Balance"),
    gr.Number(label="Number of Products (1 or 2)"),
    gr.Dropdown(choices=["0", "1"], label="Has Credit Card (0=No, 1=Yes)"),
    gr.Dropdown(choices=["0", "1"], label="Is Active Member (0=No, 1=Yes)"),
    gr.Number(label="Estimated Salary"),
]

outputs = [
    gr.Textbox(label="Churn Prediction"),
    gr.Image(label="Prediction Probability Plot"),
    gr.Image(label="Churn Distribution in Dataset"),
]

iface = gr.Interface(
    fn=predict_churn,
    inputs=inputs,
    outputs=outputs,
    css=".gradio-container {background-color: #f0f8ff; font-size: 18px; color: #333333; font-family: 'Arial'; padding: 20px;}",
    title="Customer Churn Prediction with Probability & Churn Distribution",
    description="Enter customer details to predict churn likelihood, see prediction probability, and view churn distribution.",
)

# Launch the Gradio app
iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://93ccb70a36ab25ca39.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from imblearn.under_sampling import RandomUnderSampler
import gradio as gr

# Load the dataset
df = pd.read_csv('https://github.com/YBI-Foundation/Dataset/raw/refs/heads/main/Bank%20Churn%20Modelling.csv')

# Preprocessing the dataset
df.set_index('CustomerId', inplace=True)
df.drop('Surname', axis=1, inplace=True)

# Encoding categorical variables
df['Geography'] = df['Geography'].replace({'France': 2, 'Germany': 1, 'Spain': 0})
df['Gender'] = df['Gender'].replace({'Male': 0, 'Female': 1})
df['Num Of Products'] = df['Num Of Products'].replace({1: 0, 2: 1, 3: 1, 4: 1})

# Creating a new feature for customers with zero bank balance
df['zero_balance'] = np.where(df['Balance'] == 0, 1, 0)

# Splitting the data into features and target variable
X = df.drop('Churn', axis=1)
y = df['Churn']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Handling imbalanced data using random undersampling
rus = RandomUnderSampler(random_state=42)
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

# Training the SVM model
best_svm_model = SVC(C=1, gamma=0.1, kernel="rbf", class_weight="balanced", probability=True)
best_svm_model.fit(X_train_rus, y_train_rus)

# Function to plot churn distribution
def plot_churn_distribution():
    plt.figure(figsize=(6, 4))
    sns.countplot(x='Churn', data=df, palette='Set2')
    plt.title('Churn Distribution')
    plt.xlabel('Churn (0 = Not Churn, 1 = Churn)')
    plt.ylabel('Count')
    plt.tight_layout()
    plt.savefig('churn_distribution.png')
    return 'churn_distribution.png'

# Function to make predictions based on user input
def predict_churn(CreditScore, Geography, Gender, Age, Tenure, Balance, NumOfProducts, HasCreditCard, IsActiveMember, EstimatedSalary):
    # Input validation
    if not (0 <= CreditScore <= 950):
        return "Error: Credit Score must be between 0 and 950.", None, None
    if not (0 <= Age <= 120):
        return "Error: Age must be between 0 and 120.", None, None
    if not (0 <= Tenure <= 40):
        return "Error: Tenure must be between 0 and 40.", None, None
    if Balance < 0:
        return "Error: Balance cannot be negative.", None, None
    if NumOfProducts not in [1, 2]:
        return "Error: Number of Products must be 1 or 2.", None, None

    try:
        # Prepare the input data as a DataFrame
        input_data = pd.DataFrame({
            'CreditScore': [CreditScore],
            'Geography': [Geography],
            'Gender': [Gender],
            'Age': [Age],
            'Tenure': [Tenure],
            'Balance': [Balance],
            'Num Of Products': [NumOfProducts],
            'Has Credit Card': [HasCreditCard],
            'Is Active Member': [IsActiveMember],
            'Estimated Salary': [EstimatedSalary],
            'zero_balance': [1 if Balance == 0 else 0]
        })

        # Encode categorical variables
        input_data['Geography'] = input_data['Geography'].replace({'France': 2, 'Germany': 1, 'Spain': 0})
        input_data['Gender'] = input_data['Gender'].replace({'Male': 0, 'Female': 1})
        input_data['Num Of Products'] = input_data['Num Of Products'].replace({1: 0, 2: 1})

        # Scale the input data using the same scaler
        input_data_scaled = scaler.transform(input_data)

        # Make prediction
        prediction = best_svm_model.predict(input_data_scaled)[0]
        proba = best_svm_model.predict_proba(input_data_scaled)[0][1]  # Probability of churn

        # Plot probability as a bar
        plt.figure(figsize=(5, 3))
        plt.bar(['Not Churn', 'Churn'], [1-proba, proba], color=['green', 'red'])
        plt.ylim(0, 1)
        plt.title('Churn Prediction Probability')
        plt.xlabel('Prediction')
        plt.ylabel('Probability')
        plt.tight_layout()
        plt.savefig('churn_probability.png')

        # Return the prediction result
        if prediction == 1:
            return "The customer is likely to churn.", 'churn_probability.png', plot_churn_distribution()
        else:
            return "The customer is not likely to churn.", 'churn_probability.png', plot_churn_distribution()
    except Exception as e:
        return f"Error in prediction: {e}", None, None

# Gradio interface
inputs = [
    gr.Number(label="Credit Score"),
    gr.Dropdown(choices=["France", "Germany", "Spain"], label="Geography"),
    gr.Dropdown(choices=["Male", "Female"], label="Gender"),
    gr.Number(label="Age"),
    gr.Number(label="Tenure"),
    gr.Number(label="Balance"),
    gr.Number(label="Number of Products (1 or 2)"),
    gr.Dropdown(choices=["0", "1"], label="Has Credit Card (0=No, 1=Yes)"),
    gr.Dropdown(choices=["0", "1"], label="Is Active Member (0=No, 1=Yes)"),
    gr.Number(label="Estimated Salary"),
]

outputs = [
    gr.Textbox(label="Churn Prediction"),
    gr.Image(label="Prediction Probability Plot"),
    gr.Image(label="Churn Distribution in Dataset"),
]

iface = gr.Interface(
    fn=predict_churn,
    inputs=inputs,
    outputs=outputs,
    css=".gradio-container {background-color: #f0f8ff; font-size: 18px; color: #333333; font-family: 'Arial'; padding: 20px;}",
    title="Customer Churn Prediction with Probability & Churn Distribution",
    description="Enter customer details to predict churn likelihood, see prediction probability, and view churn distribution.",
)

# Launch the Gradio app
iface.launch(share=True)


ModuleNotFoundError: No module named 'gradio'